###Global Variables

In [ ]:
# The path to where the COCO dataset will be downloaded and stored
DRIVE_DATA: str = "OlfactionDiffusion/Data"
# The path to the allowed aroma descriptors file
DRIVE_AROMA_DATA: str = "OlfactionDiffusion/Data/aroma_data"
# The path to store the COCO dattaset annotation file
ANNOTATION_FILE: str = "annotations/instances_val2017.json"
# Which OpenAI model to use
GPT_MODEL: str = "gpt-4o-mini"
# Your OpenAI API Key from here: https://platform.openai.com/api-keys
OPENAI_KEY: str = "<YOUR_OPENAI_KEY>"

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%pwd
%ls

###Download and Extract COCO Dataset

In [ ]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

"""
Uncomment which datasets you would like to download and extract.
Note that to train, you should download the 'Train' dataset, but we uncomment the
val set because it is smaller for demonstration purposes.
"""
## Train
# !wget http://images.cocodataset.org/zips/train2017.zip
# !unzip train2017.zip
# !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
# !unzip annotations_trainval2017.zip

# Test
# !wget http://images.cocodataset.org/zips/test2017.zip
# !unzip test2017.zip
# !wget http://images.cocodataset.org/annotations/annotations_test2017.zip
# !unzip annotations_test2017.zip

# Val
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_val2017.zip
!unzip annotations_val2017.zip

## Top-level imports and COCO variables

In [ ]:
import pycocotools.coco as coco
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io as io
import PIL.Image as Image
import matplotlib.patches as patches
import openai
import base64


annotation_file = ANNOTATION_FILE
coco_annotation = coco.COCO(annotation_file)
categories = coco_annotation.loadCats(coco_annotation.getCatIds())

## Verify COCO dataset was downloaded and setup correctly

In [ ]:
def check_coco_setup(debug:bool=True):
    category_names = [category['name'] for category in categories]
    category_ids = coco_annotation.getCatIds(catNms=category_names)
    image_ids = coco_annotation.getImgIds(catIds=category_ids[0])
    image_data = coco_annotation.loadImgs(image_ids)

    image_id = image_ids[0]
    image_info = coco_annotation.loadImgs([image_id])[0]
    image_path = 'train2017/' + image_info['file_name']
    if debug:
        print(f"category ids: {category_ids}")
        print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))
        print(f"image path: {image_path}")
        print(f"image ids: {image_ids}")
        print(f"image info: {image_info}")

    image = io.imread(image_path)
    annotation_ids = coco_annotation.getAnnIds(imgIds=image_info['id'], catIds=category_ids, iscrowd=None)
    annotations = coco_annotation.loadAnns(annotation_ids)

    plt.imshow(image)
    plt.show()

    # Draw boxes and add label to each box
    image = Image.open(image_path)
    fig, ax = plt.subplots()
    for ann in annotations:
        if debug:
            print(ann)
            print(ann['bbox'])
        box = ann['bbox']
        bb = patches.Rectangle((box[0],box[1]), box[2],box[3], linewidth=2, edgecolor="blue", facecolor="none")
        ax.add_patch(bb)

    ax.imshow(image)

## Verify OpenAI API was initialized correctly

In [ ]:
def check_openai_setup():
    client = openai.OpenAI(api_key=OPENAI_KEY)
    category_names = [category['name'] for category in categories]
    category_ids = coco_annotation.getCatIds(catNms=category_names)
    image_ids = coco_annotation.getImgIds(catIds=category_ids[0])
    image_data = coco_annotation.loadImgs(image_ids)

    image_id = image_ids[0]
    image_info = coco_annotation.loadImgs([image_id])[0]
    image_path = 'val2017/' + image_info['file_name']

    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # Getting the base64 string
    base64_image = encode_image(image_path)

    # Send the request to the API
    response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": [
                        {"type": "text",
                        "text": "You are a cool image analyst.  Your goal is to describe what is in the image provided as a file."
                        }
                    ],
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type":"text",
                            "text": "What is in this image?"
                        },
                        {
                            "type": "image_url",
                            "image_url":
                                {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                        }
                    ]
                }
            ],
            max_tokens=300
        )
    print(f"response: {response}")
    # Extract the description
    description = response.choices[0].message.content
    print(f"Desription: {description}")
    assert len(description) > 0, "ERROR: OPENAI API RESPONSE FAILED"

## Pull the aroma descriptors CSV data from drive

In [ ]:
def load_aroma_descriptors():
    aroma_df: pd.DataFrame = pd.read_csv(f"/content/drive/My Drive/{DRIVE_AROMA_DATA}/aroma_list.csv")
    return aroma_df["Aroma"]

#Initialize and Setup

In [ ]:
check_coco_setup()
check_openai_setup()
aromas: list = load_aroma_descriptors()

## Analyze an input image and generate expected aromas according to the scene and objects

In [ ]:
def get_descriptors_from_image(image_path: str, aromas: list):
    client = openai.OpenAI(api_key=OPENAI_KEY)

    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # Getting the base64 string
    base64_image = encode_image(image_path)

    # Send the request to the API
    response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": [
                        {"type": "text",
                        "text": f"You are an olfactory expert. Describe the aromas that could be coming from the image provided in the file. You may only use the following aromas as descriptors: {aromas}. Give only the aroma descriptors as a list in your response and give no other text. Example: ['Woody', 'Musky', 'Fruity']."
                        }
                    ],
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type":"text",
                            "text": "What aromas may be emitted from the scene in this image?"
                        },
                        {
                            "type": "image_url",
                            "image_url":
                                {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                        }
                    ]
                }
            ],
            max_tokens=300
        )
    print(f"response: {response}")
    # Extract the description
    descriptors = response.choices[0].message.content
    print(f"Desription: {descriptors}")
    assert len(list(descriptors)) > 0, "ERROR: OPENAI API RESPONSE FAILED"
    return descriptors

## Training loop

In [ ]:
from PIL import Image
import time
import json
import random


# Define a class object to store corresponding data together to serialize training samples
class OlfactionVec:
    image_id: str
    image_cat: str
    image_cat_id: str
    image_info: str
    image_path: str
    ann_bbox: list
    ann_id: str
    ann_cat: str
    all_img_labels: list
    descriptors: list
    aroma_vec: list

    def __init__(self, image_id:str):
        self.image_id = image_id


# Init list of training samples
olfactionVectors: list[OlfactionVec] = []


# Scrape the COCO dataset and build the olfaction-vision-language data
def scrape_loop(get_random_n:int=1000, debug:bool=True):
    # Define category params
    category_names = [category['name'] for category in categories]
    category_ids = coco_annotation.getCatIds(catNms=category_names)

    # Define image ids
    if get_random_n > 0:
        # Get all image IDs
        image_ids = coco_annotation.getImgIds()
        # Randomly sample 1000 image IDs
        image_ids = random.sample(image_ids, get_random_n)
    else:
        image_ids = coco_annotation.getImgIds()

    # Assign cat ids according to imag ids
    for cat_idx in range(0, len(category_ids)):
        image_data = coco_annotation.loadImgs(image_ids)

    # Generate olfactory descriptors for images
    for i in range(0, len(image_ids)):
        image_id = image_ids[i]
        image_info = coco_annotation.loadImgs([image_id])[0]
        if debug:
            print(f"image info: {image_info}")
        image_path = 'val2017/' + image_info['file_name']

        # descriptors: str = get_descriptors_from_image(image_path, aromas=aromas)
        try:
            descriptors: str = get_descriptors_from_image(image_path, aromas=aromas)
        except RateLimitError:
            print("\n\tRATE LIMIT OF OPENAI API ENCOUNTERED")
            break
        descriptors: list = descriptors.split(", ")
        descriptors: list = [descriptor.replace("[", "").replace("]", "").replace("'","").lower() for descriptor in descriptors]
        print(descriptors)
        aroma_vec: list = []
        for aroma in aromas:
            if aroma in descriptors:
                aroma_vec.append(1)
            else:
                aroma_vec.append(0)
        olfactionVec: OlfactionVec = OlfactionVec(image_id=image_id)
        olfactionVec.image_info = image_info
        olfactionVec.image_path = image_path
        olfactionVec.descriptors = descriptors
        olfactionVec.aroma_vec = aroma_vec
        olfactionVec.image_cat_id = category_ids[cat_idx]
        olfactionVec.image_cat = category_names[cat_idx]
        olfactionVectors.append(olfactionVec)

        # For quick visual checking
        if debug:
            print(f"aroma vec: {aroma_vec}")
            print(f"category ids: {category_ids}")
            print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))
            print(f"image path: {image_path}")
            print(f"image ids: {image_ids}")
            print(f"image info: {image_info}")

        image = io.imread(image_path)
        annotation_ids = coco_annotation.getAnnIds(imgIds=image_info['id'], catIds=category_ids, iscrowd=None)
        annotations = coco_annotation.loadAnns(annotation_ids)
        all_img_labels: list = []

        olfactionVec.all_img_labels = all_img_labels
        diffusion_list: list = [k for k in olfactionVectors]
        json_string1 = json.dumps(diffusion_list, default=lambda obj: obj.__dict__)

        # Save a checkpoint of the data
        with open("diffusion_list_checkpoint.json", "w") as file:
            json.dump(json_string1, file)

        print(f"\n\n\n{float(i / len(image_ids) * 100)} % complete\n\n")

        # Pause so we don't get rate-limited by OpenAI
        time.sleep(12)


## Main driver code: build the dataset

In [ ]:
# Start training
scrape_loop(get_random_n=3000,debug=True)

# Serialize dataset
diffusion_list: list = [k for k in olfactionVectors]
json_string_final = json.dumps(diffusion_list, default=lambda obj: obj.__dict__)

# Export data to local dir
with open("diffusion_list_final.json", "w") as file:
    json.dump(json_string_final, file)

# Visual check
image_paths: list = [k.image_path for k in olfactionVectors]
aroma_vecs: list = [k.aroma_vec for k in olfactionVectors]
cat_labels: list = [k.image_cat_id for k in olfactionVectors]
all_img_labels: list = [k.all_img_labels[:5] for k in olfactionVectors]
print(f"image_paths: {image_paths}")
print(f"aroma_vecs: {aroma_vecs}")
print(f"cat_labels: {cat_labels}")
print(f"all_img_labels: {all_img_labels}")

## Optionally load the json file into the variables needed for training new models

In [ ]:
import json


try:
    with open('diffusion_list.json', 'r') as file:
        data_dict = json.load(file)
    print(f"data_dict: {data_dict}")
    data_dict = json.loads(data_dict)
    print(f"data_dict: {data_dict}")
    image_paths = [item['image_path'] for item in data_dict]
    aroma_vecs = [item['aroma_vec'] for item in data_dict]
    cat_labels = [item['image_cat_id'] for item in data_dict]

except FileNotFoundError:
    print("Error: 'data.json' not found. Please ensure the file exists.")
except json.JSONDecodeError:
    print("Error: Could not decode JSON from 'data.json'. Check file format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

print(f"image_paths: {image_paths}")
print(f"aroma_vecs: {aroma_vecs}")
print(f"cat_labels: {cat_labels}")

image_paths = image_paths[:1000]
aroma_vecs = aroma_vecs[:1000]
cat_labels = cat_labels[:1000]